# Test UR Robot

In [1]:
import time
import rtde_control
import rtde_receive
import numpy as np
import yaml
import zmq
from interface.protobuf import robot_pb2

In [2]:

# Set up the RTDE connection
with open("config/address.yaml", "r") as f:
    robot_ip = yaml.load(f.read(), Loader=yaml.Loader)["robot_ip"]
rtde_r = rtde_receive.RTDEReceiveInterface(robot_ip)
# rtde_c = rtde_control.RTDEControlInterface(robot_ip)

In [4]:
class RobotPublisher:
    def __init__(self):
        self.context = zmq.Context()
        self.socket = self.context.socket(zmq.PUB)
        self.socket.bind("tcp://*:5555")
        
        self.robot = robot_pb2.Robot()

    def publish_robot_data(self, joint_angles: list, joint_velocities: list, tcp_pose: list, tcp_velocity: list, tcp_force: list):
        self.robot.joint_angles[:] = joint_angles
        self.robot.joint_velocities[:] = joint_velocities
        self.robot.tcp_pose[:] = tcp_pose
        self.robot.tcp_velocity[:] = tcp_velocity
        self.robot.tcp_force[:] = tcp_force
        
        self.socket.send(self.robot.SerializeToString())

## Robot State Visualization

This part is to visualize the robot state when human teaches the robot.

In [5]:
def read_robot_data(rtde_r):
    joint_angles = rtde_r.getActualQ()
    joint_velocities = rtde_r.getActualQd()
    tcp_pose = rtde_r.getActualTCPPose()
    tcp_velocity = rtde_r.getActualTCPSpeed()
    tcp_force = rtde_r.getActualTCPForce()
    return joint_angles, joint_velocities, tcp_pose, tcp_velocity, tcp_force

def main(fps=100):
    robot_publisher = RobotPublisher()
    start_time = time.time()
    try:
        while True:
            start_time = time.time()
            
            joint_angles, joint_velocities, tcp_pose, tcp_velocity, tcp_force = read_robot_data(rtde_r)
            robot_publisher.publish_robot_data(joint_angles, joint_velocities, tcp_pose, tcp_velocity, tcp_force)
            
            elapsed_time = time.time() - start_time
            sleep_time = max(1.0 / fps - elapsed_time, 0)
            time.sleep(sleep_time)
    except KeyboardInterrupt:
        pass

main(60)

RTDEReceiveInterface boost system Exception: (asio.misc:2) End of file
